In [13]:
import os, time, csv, glob, shutil
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# For compilation
import pandas as pd

# ---------------- CONFIG ----------------
DOWNLOAD_DIR = os.path.abspath("./downloads")  # downloaded file target
BASE_URL = "https://appv2.loveinstore.com/Home"
MAX_WAIT_SECS = 45
HEADLESS = True

# ✅ --- PUT YOUR LOGIN CREDENTIALS HERE ---
USERNAME = "MISDBRPharma-01"
PASSWORD = "Mis@LIS@2024"
# -----------------------------------------

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# =========================
# CHROME (headless) + downloads
# =========================
chrome_options = Options()
if HEADLESS:
    chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--ignore-ssl-errors=yes")

prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, MAX_WAIT_SECS)
actions = ActionChains(driver)

# =========================
# HELPERS
# =========================
def js_click(el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(locator, timeout=40, attempts=3):
    for attempt in range(attempts):
        el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))
        try:
            el.click()
            return el
        except Exception:
            try:
                js_click(el)
                return el
            except StaleElementReferenceException:
                if attempt == attempts - 1:
                    raise
                time.sleep(0.3)
    raise TimeoutException(f"Could not click {locator}")

def switch_into_frame_containing(locator, timeout=30):
    driver.switch_to.default_content()
    try:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
        return False
    except TimeoutException:
        pass
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    for fr in frames:
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame(fr)
            WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
            return True
        except Exception:
            continue
    driver.switch_to.default_content()
    return False

def wait_for_url_contains(part, timeout=40):
    WebDriverWait(driver, timeout).until(lambda d: part in d.current_url)

def set_date_input(input_el, value_ddmmyyyy):
    driver.execute_script("arguments[0].removeAttribute('readonly');", input_el)
    input_el.clear()
    input_el.click()
    input_el.send_keys(value_ddmmyyyy)
    actions.move_by_offset(0, 120).click().perform()

def find_first(locators, timeout_each=20):
    last_err = None
    for loc in locators:
        try:
            return WebDriverWait(driver, timeout_each).until(EC.presence_of_element_located(loc))
        except Exception as e:
            last_err = e
    raise last_err

def wait_for_download(dir_path, timeout=240):
    start = time.time()
    seen = set(glob.glob(os.path.join(dir_path, "*")))
    while time.time() - start < timeout:
        time.sleep(1)
        current = set(glob.glob(os.path.join(dir_path, "*")))
        new_files = list(current - seen)
        if new_files:
            while any(p.endswith(".crdownload") for p in new_files):
                time.sleep(1)
                current = set(glob.glob(os.path.join(dir_path, "*")))
                new_files = list(current - seen)
            return max(new_files, key=os.path.getmtime)
    return None

def parse_table(file_path):
    lower = file_path.lower()
    if lower.endswith(".csv"):
        try:
            return pd.read_csv(file_path, encoding="utf-8")
        except Exception:
            try:
                return pd.read_csv(file_path, encoding="utf-8-sig")
            except Exception:
                return None
    elif lower.endswith(".xlsx") or lower.endswith(".xls"):
        try:
            return pd.read_excel(file_path)
        except Exception:
            return None
    else:
        try:
            return pd.read_csv(file_path, engine="python")
        except Exception:
            return None

# =========================
# SCRIPT START TIME
# =========================
script_start_time = time.time()

# =========================
# MAIN FLOW
# =========================
frames = []

try:
    print("Opening site…")
    driver.get(BASE_URL)

    # --- Login ---
    print("Logging in…")
    username_field = wait.until(EC.presence_of_element_located((By.ID, "UserName")))
    password_field = wait.until(EC.presence_of_element_located((By.ID, "Password")))
    login_button = wait.until(EC.element_to_be_clickable((By.ID, "sign")))

    driver.execute_script("arguments[0].removeAttribute('readonly');", username_field)
    driver.execute_script("arguments[0].removeAttribute('readonly');", password_field)
    username_field.clear()
    username_field.send_keys(USERNAME)
    password_field.clear()
    password_field.send_keys(PASSWORD)
    login_button.click()

    wait.until(EC.any_of(
        EC.presence_of_element_located((By.XPATH, "//aside|//div[contains(@class,'sidebar')]")),
        EC.presence_of_element_located((By.XPATH, "//a[contains(.,'Logout') or contains(@href,'Logout')]"))
    ))
    time.sleep(1.0)

    print("Opening Data Management…")
    safe_click((
        By.XPATH,
        "//a[contains(@href,'/Admin/DataManagement/Index') and .//span[contains(text(),'Data Management')]]"
    ), timeout=40)

    print("Opening Store Action…")
    StoreAction_locators = [
        (By.XPATH, "//a[contains(.,'Store Action') and not(contains(@href,'PrimaryShelfStock'))]"),
        (By.XPATH, "//a[contains(@href,'/Store Action') and contains(.,'Store Action')]"),
    ]
    clicked = False
    for loc in StoreAction_locators:
        try:
            safe_click(loc, timeout=40)
            clicked = True
            break
        except Exception:
            continue
    if not clicked:
        raise TimeoutException("Could not open StoreAction (all locators failed).")

    print("Opening Primary Shelf Stock…")
    monthly_locators = [
        (By.XPATH, "//a[contains(@href,'/StoreActionV2/PrimaryShelfStockV2') and contains(.,'Primary Shelf Stock')]"),
        (By.XPATH, "//a[contains(.,'Primary Shelf Stock')]"),
    ]
    clicked = False
    for loc in monthly_locators:
        try:
            el = WebDriverWait(driver, 40).until(EC.presence_of_element_located(loc))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            try:
                el.click()
            except Exception:
                js_click(el)
            clicked = True
            break
        except Exception:
            continue
    if not clicked:
        raise TimeoutException("Could not open Primary Shelf Stock (all locators failed).")

    try:
        wait_for_url_contains("StoreAction", timeout=40)
    except Exception:
        pass

    target_dropdown = (By.XPATH, "//*[@id='Sdatewise' or @name='Sdatewise' or @id='select2-Sdatewise-container']")
    switch_into_frame_containing(target_dropdown, timeout=30)
    WebDriverWait(driver, 40).until(EC.presence_of_element_located(target_dropdown))

    print("Selecting Datewise…")
    try:
        select2_container = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@id='select2-Sdatewise-container']"))
        )
        select2_container.click()
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@class,'select2-search__field')]"))
        )
        search_box.clear()
        search_box.send_keys("Datewise")
        time.sleep(0.3)
        search_box.send_keys(Keys.ENTER)
    except TimeoutException:
        try:
            datewise_option = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[contains(@id,'select2-Sdatewise') and normalize-space()='Datewise']"))
            )
            js_click(datewise_option)
        except Exception as e:
            print("Select2 fallback failed:", e)

    first_day_dt = datetime.today().replace(day=1)
    yesterday_dt = datetime.today() - timedelta(days=1)

    print(f"Daily iteration split into 3 parts: {first_day_dt.strftime('%d/%m/%Y')} -> {yesterday_dt.strftime('%d/%m/%Y')}")

    # ----------------------------
    # Helper function for a date range
    # ----------------------------
    def process_date_range(start_dt, end_dt, part_name):
        print(f"\n🟦 Processing {part_name}: {start_dt.strftime('%d/%m/%Y')} -> {end_dt.strftime('%d/%m/%Y')}")
        current_dt = start_dt
        day_index = 1
        while current_dt <= end_dt and current_dt <= yesterday_dt:
            try:
                day_str = current_dt.strftime("%d/%m/%Y")
                print(f"\n=== {part_name} - Day {day_index}: {day_str} ===")

                # Re-find input elements to prevent stale references
                from_input = find_first([
                    (By.ID, "Sdate"),
                    (By.NAME, "DateFrom"),
                    (By.XPATH, "//input[@placeholder='From' or @placeholder='Date From']")
                ])
                to_input = find_first([
                    (By.ID, "dateto"),
                    (By.NAME, "DateTo"),
                    (By.XPATH, "//input[@placeholder='To' or @placeholder='Date To']")
                ])

                set_date_input(from_input, day_str)
                set_date_input(to_input, day_str)
                time.sleep(1.5)

                download_dropdown_locators = [
                    (By.XPATH, "//a[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
                    (By.XPATH, "//button[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
                ]
                opened = False
                for loc in download_dropdown_locators:
                    try:
                        safe_click(loc)
                        opened = True
                        break
                    except Exception:
                        time.sleep(1)
                        continue
                if not opened:
                    print("⚠️ Could not open 'Download Now' dropdown. Skipping this day.")
                    current_dt += timedelta(days=1); day_index += 1
                    continue

                master_export_locators = [
                    (By.XPATH, "//button[@type='submit' and contains(@class,'dropdown-item') and @value='Export' and contains(.,'Export')]"),
                    (By.XPATH, "//button[contains(.,'Export') and (@type='submit' or @value='Export')]"),
                    (By.XPATH, "//a[contains(.,'Export')]"),
                ]
                exported = False
                for loc in master_export_locators:
                    try:
                        safe_click(loc)
                        exported = True
                        break
                    except Exception:
                        time.sleep(1)
                        continue
                if not exported:
                    print("⚠️ Could not click Export. Skipping this day.")
                    current_dt += timedelta(days=1); day_index += 1
                    continue

                downloaded_path = wait_for_download(DOWNLOAD_DIR, timeout=240)
                if not downloaded_path:
                    print(f"ℹ️ No file downloaded for {day_str}. Skipping.")
                    current_dt += timedelta(days=1); day_index += 1
                    continue

                try:
                    if os.path.getsize(downloaded_path) < 5:
                        print(f"ℹ️ Empty file for {day_str}. Skipping.")
                        os.remove(downloaded_path)
                        current_dt += timedelta(days=1); day_index += 1
                        continue
                except Exception:
                    pass

                df = parse_table(downloaded_path)
                if df is None or df.empty:
                    print(f"ℹ️ No data inside file for {day_str}. Skipping.")
                    try:
                        os.remove(downloaded_path)
                    except Exception: pass
                    current_dt += timedelta(days=1); day_index += 1
                    continue

                df.insert(0, "PulledDate", current_dt.strftime("%Y-%m-%d"))
                frames.append(df)
                print(f"✅ Loaded {len(df)} rows for {day_str}")

                try:
                    os.remove(downloaded_path)
                except Exception as e:
                    print(f"⚠️ Could not delete temp file {downloaded_path}: {e}")

                time.sleep(1.5)

            except Exception as e:
                print(f"⚠️ Unexpected error on {current_dt.strftime('%d/%m/%Y')}: {e}. Skipping this day.")

            current_dt += timedelta(days=1)
            day_index += 1

    # =========================
    # Run the three parts
    # =========================
    part1_start = first_day_dt
    part1_end = first_day_dt.replace(day=10)
    part2_start = first_day_dt.replace(day=11)
    part2_end = first_day_dt.replace(day=20)
    part3_start = first_day_dt.replace(day=21)
    part3_end = yesterday_dt

    process_date_range(part1_start, part1_end, "Part 1 (Days 1–10)")
    process_date_range(part2_start, part2_end, "Part 2 (Days 11–20)")
    process_date_range(part3_start, part3_end, "Part 3 (Days 21–End)")

    # =========================
    # WRITE ONE FINAL CSV
    # =========================
    if frames:
        compiled = pd.concat(frames, ignore_index=True)
        compiled_name = f"PrimaryShelfStock_compiled_{first_day_dt.strftime('%Y%m%d')}_{yesterday_dt.strftime('%Y%m%d')}.csv"
        compiled_path = os.path.join(DOWNLOAD_DIR, compiled_name)
        compiled.to_csv(compiled_path, index=False, encoding="utf-8-sig")
        print(f"📦 Compiled CSV created: {compiled_path} (rows: {len(compiled)})")
    else:
        print("⚠️ No readable files to compile. Skipped creating compiled CSV.")

except Exception as e:
    print("❌ Automation failed:", e)
finally:
    try:
        driver.quit()
    except Exception:
        pass

    script_end_time = time.time()
    total_runtime = script_end_time - script_start_time
    minutes, seconds = divmod(total_runtime, 60)
    print(f"⏱ Total script runtime: {int(minutes)} min {int(seconds)} sec")


Opening site…
Logging in…
Opening Data Management…
Opening Store Action…
Opening Primary Shelf Stock…
Selecting Datewise…
Daily iteration split into 3 parts: 01/11/2025 -> 03/11/2025

🟦 Processing Part 1 (Days 1–10): 01/11/2025 -> 10/11/2025

=== Part 1 (Days 1–10) - Day 1: 01/11/2025 ===
ℹ️ No file downloaded for 01/11/2025. Skipping.

=== Part 1 (Days 1–10) - Day 2: 02/11/2025 ===
⚠️ Unexpected error on 02/11/2025: Message: 
Stacktrace:
	GetHandleVerifier [0x0xe3ba83+63395]
	GetHandleVerifier [0x0xe3bac4+63460]
	(No symbol) [0x0xc82113]
	(No symbol) [0x0xcca85e]
	(No symbol) [0x0xccabfb]
	(No symbol) [0x0xd12f92]
	(No symbol) [0x0xcef3f4]
	(No symbol) [0x0xd107ba]
	(No symbol) [0x0xcef1a6]
	(No symbol) [0x0xcbe7b2]
	(No symbol) [0x0xcbf654]
	GetHandleVerifier [0x0x10b8883+2672035]
	GetHandleVerifier [0x0x10b3cba+2652634]
	GetHandleVerifier [0x0xe62bca+223466]
	GetHandleVerifier [0x0xe52cb8+158168]
	GetHandleVerifier [0x0xe5978d+185517]
	GetHandleVerifier [0x0xe43b78+96408]
	GetHandle